In [1]:
!pip install elasticsearch

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': '7BHkxSp9QmWEUMUtBbmB1A',
 'name': 'f520cb9bb5e1',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-05-28T10:06:37.834829258Z',
             'build_flavor': 'default',
             'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.1.0',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.18.0',
             'number': '9.0.2'}}


In [ ]:
es.indices.delete(index='')

ObjectApiResponse({'acknowledged': True})

In [55]:
#list index
try:
    indices = list(es.indices.get_alias(index="*").keys())
    
    indices_reais = [idx for idx in indices if not idx.startswith('.')]
    print("Índices encontrados:", indices_reais)
except Exception as e:
    print(f"Erro ao tentar obter os índices: {e}")

Índices encontrados: ['test_index', 'main_index']


In [40]:
#list documents
index_name = 'test_index'

try:
    response = es.count(index=index_name)
    total = response['count']
    print(f"Total documents in '{index_name}': {total}")
except Exception as e:
    print(f"Error counting documents: {e}")

Total documents in 'test_index': 56


In [23]:
# Assume 'es' is your configured Elasticsearch client object

size = 10
search_term = "ksWj4pcB2sAPyDwPNrA6"
index_name = 'test_index'

# Correctly structured query body
# This part remains the same as the previous fix.
body = {
    "size": size,
    # "_source": {
    #                 "excludes": ["document.body"]
    #             },
    "query": {
        "query_string": {
            "query": search_term,
        }
    }
}

try:
    # Use the 'body' parameter, which is correct for your library version.
    response = es.search(index=index_name, body=body) 
    pprint(response)
    
    documents = [hit["_source"] for hit in response["hits"]["hits"]]
    print(f"Found {len(documents)} documents matching '{search_term}' in '{index_name}'.")

    print(documents)
except Exception as e:
    print(f"Error searching documents: {e}")

ObjectApiResponse({'took': 12, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})
Found 0 documents matching 'ksWj4pcB2sAPyDwPNrA6' in 'test_index'.
[]


In [15]:
dict(documents[0])

{'id': '729397230',
 'document': {'title': 'Apelação Cível: AC 10003661320178260459 SP 1000366-13.2017.8.26.0459',
  'body': 'PODER JUDICIÁRIO TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO Registro: 2019.0000525018 ACÓRDÃO Vistos, relatados e discutidos estes autos de Apelação Cível nº 1000366-13.2017.8.26.0459, da Comarca de Pitangueiras, em que são apelantes TEREOS AÇUCAR E ENERGIA BRASIL S/A e ANDRADE AÇUCAR E ÁLCOOL S/A, é apelado ESTADO DE SÃO PAULO. ACORDAM , em sessão permanente e virtual da 2ª Câmara Reservada ao Meio Ambiente do Tribunal de Justiça de São Paulo, proferir a seguinte decisão: Deram provimento ao recurso. V. U. , de conformidade com o voto do relator, que integra este acórdão. O julgamento teve a participação dos Desembargadores MIGUEL PETRONI NETO (Presidente) e LUIS FERNANDO NISHI. São Paulo, 2 de julho de 2019. Paulo Alcides Relator Assinatura Eletrônica PODER JUDICIÁRIO TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO 2ª CÂMARA RESERVADA AO MEIO AMBIENTE VOTO Nº 37053

In [7]:
import os
print(f"DIRETÓRIO DE TRABALHO DO NOTEBOOK: {os.getcwd()}")

DIRETÓRIO DE TRABALHO DO NOTEBOOK: c:\Users\hugom\OneDrive\Documentos\codes\tp1-RI\SearchEngine\exploring_data


In [54]:
id = "899371419"

query = {
            "query": {
                "term": {
                    "_id": id # Em muitos sistemas (como Elasticsearch), o campo de ID padrão é '_id'
                }
            },
            "size": 1 # Só precisamos saber se existe, então 1 resultado é suficiente
        }
        
        # Executa a busca
search_result = es.search(index="test_index", body=query)

pprint(dict(search_result["hits"]["hits"][0]["_source"]))

{'document': {'body': 'Poder Judiciário TRIBUNAL REGIONAL FEDERAL DA 4ª REGIÃO '
                      'Agravo de Instrumento Nº 5002871-65.2020.4.04.0000 /RS '
                      'RELATOR: Juiz Federal JULIO GUILHERME BEREZOSKI '
                      'SCHATTSCHNEIDER RELATÓRIO Trata-se de agravo de '
                      'instrumento contra "(...) Por outro lado, indefiro, por '
                      'ora, o pedido de requisição dos honorários contratuais, '
                      'os quais somente poderão ser requisitados por ocasião '
                      'da requisição do crédito da parte exequente, uma vez '
                      'que aqueles estão vinculados a este. Ademais, ainda em '
                      'relação à reserva dos honorários contratuais, observo '
                      'que, nos termos do artigo 607 do Código Civil ,"O '
                      'contrato de prestação de serviço acaba com a morte de '
                      'qualquer das partes". Dessa forma, par